In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
from scipy.stats import linregress

# Import API key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
data = "Project_data.csv"

# Range of latitudes and longitudes
df_data = pd.read_csv(data)
df_data['Lat'] = ""
df_data['Lng'] = ""
df_data

,Unnamed: 0,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
0,0,2010,Arizona,15.916712,33.981276,0.025903,0.043353,1.047033,1.999018,0.365168,0.697395,1617,6407172,-0.004629,,
1,1,2010,Arkansas,10.004749,23.138889,0.027650,0.041337,1.292494,2.168403,0.454634,0.605972,2220,2921964,0.026372,,
2,2,2010,California,12.247269,23.389085,0.025669,0.037684,0.720866,1.467768,0.344747,0.572430,5776,37319502,0.209190,,
3,3,2010,Colorado,16.171138,33.061321,0.022955,0.037774,1.028090,4.286439,0.365696,0.685613,2694,5047349,0.050741,,
4,4,2010,Connecticut,9.739028,21.720350,0.027474,0.039643,0.949623,2.499313,0.279957,0.400370,418,3579114,-0.066271,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,207,2015,Texas,9.252533,19.689344,0.024904,0.037034,0.258306,1.003250,0.207629,0.375761,8304,27470056,0.339158,,
208,208,2015,Utah,16.981693,31.588933,0.027734,0.041411,0.251122,0.735968,0.360339,0.670119,605,2981835,-0.056657,,
209,209,2015,Virginia,5.442843,12.570026,0.029290,0.041366,0.387699,0.957752,0.267784,0.368798,597,8361808,-0.057068,,
210,210,2015,Washington,9.672585,22.670833,0.021468,0.030310,0.813735,1.611310,0.203479,0.281574,1741,7163657,0.001746,,


In [3]:
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in df_data.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['State']

    # update address key value
    params['address'] = f"{city}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    df_data.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    df_data.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
df_data.head()

,Unnamed: 0,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
0,0,2010,Arizona,15.916712,33.981276,0.025903,0.043353,1.047033,1.999018,0.365168,0.697395,1617,6407172,-0.004629,34.0489,-111.094
1,1,2010,Arkansas,10.004749,23.138889,0.027650,0.041337,1.292494,2.168403,0.454634,0.605972,2220,2921964,0.026372,35.2011,-91.8318
2,2,2010,California,12.247269,23.389085,0.025669,0.037684,0.720866,1.467768,0.344747,0.572430,5776,37319502,0.209190,36.7783,-119.418
3,3,2010,Colorado,16.171138,33.061321,0.022955,0.037774,1.028090,4.286439,0.365696,0.685613,2694,5047349,0.050741,39.5501,-105.782
4,4,2010,Connecticut,9.739028,21.720350,0.027474,0.039643,0.949623,2.499313,0.279957,0.400370,418,3579114,-0.066271,41.6032,-73.0877


In [4]:
#hemisferio norte
df_data_2010 = df_data.loc[df_data["Year"] == 2010, [
    "Year","State", "NO2 Mean", "NO2 1st Max Value", "O3 Mean", "O3 1st Max Value", "SO2 Mean", 
    "SO2 1st Max Value", "CO Mean", "CO 1st Max Value", "Wildfires", "population", "N_wildfire", "Lat", "Lng"]]
df_data_2010.head()

,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
0,2010,Arizona,15.916712,33.981276,0.025903,0.043353,1.047033,1.999018,0.365168,0.697395,1617,6407172,-0.004629,34.0489,-111.094
1,2010,Arkansas,10.004749,23.138889,0.027650,0.041337,1.292494,2.168403,0.454634,0.605972,2220,2921964,0.026372,35.2011,-91.8318
2,2010,California,12.247269,23.389085,0.025669,0.037684,0.720866,1.467768,0.344747,0.572430,5776,37319502,0.209190,36.7783,-119.418
3,2010,Colorado,16.171138,33.061321,0.022955,0.037774,1.028090,4.286439,0.365696,0.685613,2694,5047349,0.050741,39.5501,-105.782
4,2010,Connecticut,9.739028,21.720350,0.027474,0.039643,0.949623,2.499313,0.279957,0.400370,418,3579114,-0.066271,41.6032,-73.0877


In [5]:
lugar = df_data_2010[["Lat","Lng"]]
hume = df_data_2010["CO Mean"]
#print(df_hero["Max Temp"].max())

In [6]:
fig= gmaps.figure(map_type='HYBRID')
heat_layer =  gmaps.heatmap_layer(lugar,weights=hume,dissipating=False,max_intensity =1 ,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#hemisferio norte
df_data_2015 = df_data.loc[df_data["Year"] == 2015, [
    "Year","State", "NO2 Mean", "NO2 1st Max Value", "O3 Mean", "O3 1st Max Value", "SO2 Mean", 
    "SO2 1st Max Value", "CO Mean", "CO 1st Max Value", "Wildfires", "population", "N_wildfire", "Lat", "Lng"]]
df_data_2015.head()

,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
171,2015,Alabama,9.078580,21.699000,0.024000,0.038627,1.009429,4.243500,0.221027,0.419567,2074,4852347,0.018866,32.3182,-86.9023
172,2015,Alaska,11.116589,20.105087,0.014218,0.020669,6.159011,9.831638,0.405221,0.655787,771,737498,-0.048123,64.2008,-149.494
173,2015,Arizona,12.704995,27.963663,0.028144,0.044836,0.584848,1.274938,0.355645,0.625053,3819,6829676,0.108579,34.0489,-111.094
174,2015,Arkansas,8.612833,21.063736,0.024335,0.035753,1.110246,2.030220,0.306023,0.410974,1289,2978048,-0.021492,35.2011,-91.8318
175,2015,California,10.269062,20.370227,0.027551,0.040394,0.458648,1.077434,0.333131,0.555143,7365,38918045,0.290883,36.7783,-119.418


In [10]:
lugar = df_data_2015[["Lat","Lng"]]
hume = df_data_2015["CO Mean"]
#print(df_hero["Max Temp"].max())

In [12]:
fig= gmaps.figure(map_type='HYBRID')
heat_layer =  gmaps.heatmap_layer(lugar,weights=hume,dissipating=False,max_intensity =1 ,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
#hemisferio norte
df_data_2010 = df_data.loc[df_data["Year"] == 2010, [
    "Year","State", "NO2 Mean", "NO2 1st Max Value", "O3 Mean", "O3 1st Max Value", "SO2 Mean", 
    "SO2 1st Max Value", "CO Mean", "CO 1st Max Value", "Wildfires", "population", "N_wildfire", "Lat", "Lng"]]
df_data_2010.head()

,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
0,2010,Arizona,15.916712,33.981276,0.025903,0.043353,1.047033,1.999018,0.365168,0.697395,1617,6407172,-0.004629,34.0489,-111.094
1,2010,Arkansas,10.004749,23.138889,0.027650,0.041337,1.292494,2.168403,0.454634,0.605972,2220,2921964,0.026372,35.2011,-91.8318
2,2010,California,12.247269,23.389085,0.025669,0.037684,0.720866,1.467768,0.344747,0.572430,5776,37319502,0.209190,36.7783,-119.418
3,2010,Colorado,16.171138,33.061321,0.022955,0.037774,1.028090,4.286439,0.365696,0.685613,2694,5047349,0.050741,39.5501,-105.782
4,2010,Connecticut,9.739028,21.720350,0.027474,0.039643,0.949623,2.499313,0.279957,0.400370,418,3579114,-0.066271,41.6032,-73.0877


In [17]:
lugar = df_data_2010[["Lat","Lng"]]
hume = df_data_2010["SO2 Mean"]
#print(df_hero["Max Temp"].max())

In [18]:
fig= gmaps.figure(map_type='HYBRID')
heat_layer =  gmaps.heatmap_layer(lugar,weights=hume,dissipating=False,max_intensity =1 ,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
#hemisferio norte
df_data_2015 = df_data.loc[df_data["Year"] == 2015, [
    "Year","State", "NO2 Mean", "NO2 1st Max Value", "O3 Mean", "O3 1st Max Value", "SO2 Mean", 
    "SO2 1st Max Value", "CO Mean", "CO 1st Max Value", "Wildfires", "population", "N_wildfire", "Lat", "Lng"]]
df_data_2015.head()

,Year,State,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,Wildfires,population,N_wildfire,Lat,Lng
171,2015,Alabama,9.078580,21.699000,0.024000,0.038627,1.009429,4.243500,0.221027,0.419567,2074,4852347,0.018866,32.3182,-86.9023
172,2015,Alaska,11.116589,20.105087,0.014218,0.020669,6.159011,9.831638,0.405221,0.655787,771,737498,-0.048123,64.2008,-149.494
173,2015,Arizona,12.704995,27.963663,0.028144,0.044836,0.584848,1.274938,0.355645,0.625053,3819,6829676,0.108579,34.0489,-111.094
174,2015,Arkansas,8.612833,21.063736,0.024335,0.035753,1.110246,2.030220,0.306023,0.410974,1289,2978048,-0.021492,35.2011,-91.8318
175,2015,California,10.269062,20.370227,0.027551,0.040394,0.458648,1.077434,0.333131,0.555143,7365,38918045,0.290883,36.7783,-119.418


In [21]:
lugar = df_data_2015[["Lat","Lng"]]
hume = df_data_2015["SO2 Mean"]
#print(df_hero["Max Temp"].max())

In [22]:
fig= gmaps.figure(map_type='HYBRID')
heat_layer =  gmaps.heatmap_layer(lugar,weights=hume,dissipating=False,max_intensity =1 ,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))